In [42]:
import pandas as pd
import warnings
import pymongo
from datetime import datetime
# pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Setup MongoDB connection (local)
mongo_host = "localhost"
mongo_port = 27017
mongo_user = "admin"
mongo_password = "password"
auth_db = "admin"
client_mongo = pymongo.MongoClient(
    host=mongo_host,
    port=mongo_port,
    username=mongo_user,
    password=mongo_password,
    authSource=auth_db
)

# GET eur_usd monthly average
db_mongo_datalake = client_mongo.get_database("datalake")
collection_mongo = db_mongo_datalake.get_collection("stockmarket_eur_usd")
cursor = collection_mongo.find({})
df_eur_usd = pd.DataFrame(list(cursor))

df_eur_usd['Date'] = pd.to_datetime(df_eur_usd['Date'])
df_eur_usd_monthly_avg = df_eur_usd.resample('M', on='Date')['Close'].mean().reset_index()
df_eur_usd_monthly_avg = df_eur_usd_monthly_avg.rename(columns={'Close': 'EUR_USD_monthly_avg'})
# Transform date to first day of month
df_eur_usd_monthly_avg['Date_monthly'] = df_eur_usd_monthly_avg['Date'].dt.to_period('M').dt.to_timestamp()
print('df_eur_usd_monthly_avg\n', df_eur_usd_monthly_avg.head())

# GET refining margin brent
url = "https://www.ecologie.gouv.fr/sites/default/files/documents/Historique%20de%20la%20marge%20brute%20de%20raffinage%20sur%20Brent%20depuis%202015%20%28moyennes%20mensuelles%29_1.xlsx"
df = pd.read_excel(url, sheet_name=0, skiprows=1)
df = df[['Moyennes mensuelles', 'en $/b']].rename(columns={'Moyennes mensuelles': 'Date_monthly'})
print('df_refining_margin_brent\n', df.head(5))

# Keep only rows where 'Date' can be converted to a datetime (removes annual averages table)
df = df[pd.to_datetime(df['Date_monthly'], errors='coerce', format='%Y-%m-%d').notna()].copy()
df['Date_monthly'] = pd.to_datetime(df['Date_monthly'])
df['en $/liter'] = (df['en $/b']/ 158.987).round(5)
print('df_refining_margin_brent \n', df.head())

# Merge monthly EUR/USD average with refining margin data
df_merged = pd.merge(df, df_eur_usd_monthly_avg, on='Date_monthly', how='left')
df_merged['refining_margin_brent_eur_liter'] = (df_merged['en $/liter']/ df_merged['EUR_USD_monthly_avg']).round(5)
df_merged = df_merged.drop(columns=['Date', 'en $/b', 'en $/liter', 'EUR_USD_monthly_avg'])
print('df_merged\n', df_merged)



# ----- Push to MongoDB -----
db_mongo_denorm = client_mongo.get_database("denormalization")
db_mongo_denorm.drop_collection("refining_margin_brent_eur")
collection_mongo = db_mongo_denorm.get_collection("refining_margin_brent_eur")
collection_mongo.create_index([("Date", pymongo.ASCENDING)])

records = df_merged.to_dict(orient="records")
collection_mongo.insert_many(records)
print("correctly loaded refining_margin_brent_usd datas to MongoDB in denormalized collection")

df_eur_usd_monthly_avg
         Date  EUR_USD_monthly_avg Date_monthly
0 2007-01-31             1.300117   2007-01-01
1 2007-02-28             1.308825   2007-02-01
2 2007-03-31             1.325345   2007-03-01
3 2007-04-30             1.351548   2007-04-01
4 2007-05-31             1.350961   2007-05-01
df_refining_margin_brent
           Date_monthly    en $/b
0  2015-01-01 00:00:00  6.987702
1  2015-02-01 00:00:00  7.013642
2  2015-03-01 00:00:00  8.503058
3  2015-04-01 00:00:00  7.134945
4  2015-05-01 00:00:00  6.846143
df_refining_margin_brent 
   Date_monthly    en $/b  en $/liter
0   2015-01-01  6.987702     0.04395
1   2015-02-01  7.013642     0.04411
2   2015-03-01  8.503058     0.05348
3   2015-04-01  7.134945     0.04488
4   2015-05-01  6.846143     0.04306
df_merged
     Date_monthly  refining_margin_brent_eur_liter
0     2015-01-01                          0.03769
1     2015-02-01                          0.03885
2     2015-03-01                          0.04931
3     2015